# Lesson 2: Probability



## Exercise 1: Monty Hall problem

- Each toy experiment is a run of the game where each player chooses randomly a door and either wins or loses
- Repeat for 100 runs

In [7]:
import matplotlib.pyplot as plt
import numpy as np

In [67]:
np.random.seed(404)
wins = [0,0,0] # Wins of conservative, switcher, newcomer
doors = [0,0,1] # 0 is goat, 1 is car
N_runs = 10000

In [68]:

for i in range(N_runs):
    np.random.shuffle(doors) # Uses random.random() function

    pl0 = np.random.choice([0,1,2]) #Conservative 
    pl1 = np.random.choice([0,1,2]) #Switcher

    # We can already say if the conservative player won:
    if doors[pl0] == 1 :
        wins[0] += 1

    # After a goat is showed, if the switcher has the car, he loses; if he as the goat, he wins:
    if doors[pl1] == 0 : 
        wins[1] +=1

    # After a goat is showed, there are 2 doors, one wins and the other loses, indifferently from what happened before.
    # As it does not matter which goat we remove, we choose the first one in the list:
    doors = doors[:]
    doors.remove(0)

    pl2 = np.random.choice([0,1]) #Newcomer

    if doors[pl2] == 1:
        wins[2] += 1

win_perc = [x/N_runs * 100 for x in wins]

In [70]:
print('Win percentages: \n \
    Conservative: %.2f %%  \n \
    Switcher: %.2f %%  \n \
    Newcomer %.2f %%' %tuple(win_perc))


Win percentages: 
     Conservative: 34.11 %  
     Switcher: 66.24 %  
     Newcomer 49.95 %


- We obtain the expected probabilities.

## Generalization to N doors

In [71]:
np.random.seed(403)
n_doors = 3
n_open = 1

wins = [0,0,0] # Wins of conservative, switcher, newcomer

N_runs = 1000

for i in range(N_runs):
    
    doors = np.zeros(n_doors) # 0 is goat, 1 is car
    doors[0] += 1
    np.random.shuffle(doors) # Uses random.random() function
    numbers = np.array(range(n_doors)) 
    doors = doors[:,np.newaxis]
    numbers = numbers[:,np.newaxis]
    doors = np.concatenate((numbers,doors), axis=1) # now "doors" is an array of couples (every door has a label)

    pl0 = np.random.choice(range(n_doors)) #Conservative 
   # pl1 = np.random.choice(range(n_doors)) #Switcher
    pl1=2

    # We can already say if the conservative player won:
    if doors[pl0,1] == 1 :
        wins[0] += 1

    # n_open doors are opened
    doors_new = doors
    j = 0
    for i in range(n_open):
        if doors_new[0,1] == 1 or doors_new[0,0] == pl1:
            j = 1
        doors_new = np.delete(doors_new, j, 0)

    door_choice = doors_new[:,0]

    pl1_second = int(np.random.choice(door_choice)) #Switcher's second guess, while cycle to prevent him to choose the same door
    k = 0
    while pl1_second == pl1: 
         pl1_second = int(np.random.choice(door_choice))
         k += 1

    if doors[pl1_second,1] == 1 : 
        wins[1] +=1


    pl2 = int(np.random.choice(door_choice)) #Newcomer

    if doors[pl2,1] == 1:
        wins[2] += 1

win_perc = [x/N_runs * 100 for x in wins]

In [70]:
print('Win percentages: \n \
    Conservative: %.2f %%  \n \
    Switcher: %.2f %%  \n \
    Newcomer %.2f %%' %tuple(win_perc))

Win percentages: 
     Conservative: 30.00 %  
     Switcher: 51.10 %  
     Newcomer 44.30 %
